# ClassyMIDI (Ver 1.0)

## Simple, yet powerful MIDI classifier and songs names generator.

***

### This is a work in progress so please check back for updates.

***

#### Project Los Angeles

#### Tegridy Code 2021

***

In [ ]:
#@title Install dependencies
!git clone https://github.com/asigalov61/tegridy-tools
!pip install pypianoroll
!pip install texttable
!pip install fuzzywuzzy[speedup]
!pip install tqdm
!pip install pretty_midi

In [ ]:
#@title Import needed modules and create IO dirs
%cd /content/tegridy-tools/tegridy-tools
import TMIDI
import ClassyMIDI
%cd /content/
import secrets
import sys
import os
import tqdm
from tqdm import auto
from shutil import copyfile
import secrets
import pypianoroll
from pypianoroll import Multitrack
from texttable import Texttable

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from glob import glob

import pretty_midi
import pypianoroll

import numpy as np
import pickle
import pandas as pd
import scipy.stats
from collections import Counter
from functools import reduce

from copy import deepcopy

import json
import logging


%cd /content
!mkdir Input
!mkdir Output

In [ ]:
#@title Download and unzip clean_midi MIDI dataset
%cd /content/Input/
!wget http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz
!tar -xvf /content/Input/clean_midi.tar.gz

In [ ]:
#@title Print music metrics for a MIDI file
full_path_to_MIDI_file = "/content/Input/clean_midi/\"Weird Al\" Yankovic/Amish Paradise.mid" #@param {type:"string"}

PR, SIG = ClassyMIDI.get_music_metrics(full_path_to_MIDI_file, 4, 0)
X = ClassyMIDI.all_features(full_path_to_MIDI_file)

print('Printing all features...')
print('=' * 100)

print('track_programs:', X[0][0])
print('track_names:', X[0][1])
print('file_names:', X[0][2])
print('is_drum:', X[0][3])
print('=' * 100)

print('Duration:', X[0][4])
print('Number of notes:', X[0][5])
print('Occupation rate:', X[0][6])
print('Polyphony rate:', X[0][7])
print('=' * 100)

print('Highest Pitch:', X[0][8])
print('Lowest Pitch:', X[0][9])
print('Pitch Avg:', X[0][10])
print('Pitch Std Dev:', X[0][11])
print('=' * 100)

print('Highest pitch norm', X[0][12])
print('lowest pitch norm:', X[0][13])
print('pitch mode norm:', X[0][14])
print('pitch std norm:', X[0][15])
print('=' * 100)

print('number of interval:', X[0][16])
print('largest interval:', X[0][17])
print('smallest interval:', X[0][18])
print('interval mode:', X[0][19])
print('=' * 100)

print('number of interval norm:', X[0][20])
print('largest interval norm:', X[0][21])
print('smallest interval norm:', X[0][22])
print('interval mode norm:', X[0][23])
print('longest note duration:', X[0][24])
print('shortest note duration:', X[0][25])
print('=' * 100)

print('mean note duration:', X[0][26])
print('longest note duration:', X[0][27])
print('note duration std:', X[0][28])
print('longest note duration norm:', X[0][29])
print('shortest note duration norm:', X[0][30])
print('mean note duration norm:', X[0][31])
print('note duration std norm:', X[0][32])
print('=' * 100)

SIG_SIMPLE = [SIG[0], SIG[1] * 1, SIG[2], SIG[3]]
SIG_FULL = [float(X[0][4] * 1), # Duration
            float(X[0][5]),
            float(X[0][6] * 1),
            float(X[0][7] * 1),
            
            float(X[0][8]), # Highest Pitch
            float(X[0][9]),
            float(X[0][10]),
            float(X[0][11]),

            float(X[0][12] * 1), # Highest pitch norm
            float(X[0][13] * 1),
            float(X[0][14] * 1),
            float(X[0][15] * 1),

            float(X[0][16]), # number of interval
            float(X[0][17]),
            float(X[0][18]),
            float(X[0][19]),


            float(X[0][20] * 1), # number of interval norm
            float(X[0][21] * 1),
            float(X[0][22] * 1),
            float(X[0][23] * 1),
            float(X[0][24] * 1),
            float(X[0][25] * 1),

            float(X[0][26] * 1), # mean note duration
            float(X[0][27] * 1),
            float(X[0][28] * 1),
            float(X[0][29] * 1),
            float(X[0][30] * 1),
            float(X[0][31] * 1),
            float(X[0][32] * 1)]

print('Resulting simple signature:', SIG_SIMPLE)
print('Resulting simple signature length:', len(SIG_SIMPLE))
print('Resulting simple signature summ:', sum(SIG_SIMPLE))
print('=' * 100)

print('Resulting simple signature:', SIG_FULL)
print('Resulting simple signature length:', len(SIG_FULL))
print('Resulting simple signature summ:', sum(SIG_FULL))
print('=' * 100)

In [ ]:
#@title Process your dataset in Input dir
print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

files_count = 0

dataset_addr = "/content/Input"
os.chdir(dataset_addr)
filez = [y for x in os.walk(dataset_addr) for y in glob(os.path.join(x[0], '*.mid'))]

CLASS_DATA = []

print('Processing MIDI files. Please wait...')
for f in tqdm.auto.tqdm(filez):
  
  try:
    SIG_SIMPLE, SIG_FULL, ALL_F, PRL = ClassyMIDI.get_signatures(f)
    fn = os.path.basename(f)
    fnn = fn.split('.')[0]
    X = PRL.flatten()
    s = secrets.randbelow(len(X))
   
    CLASS_DATA.append([f, fnn, [], SIG_SIMPLE, SIG_FULL, ALL_F.tolist(), X[s:s+64].tolist()])
    files_count += 1

  except KeyboardInterrupt:
    break

  except:
    print('Bad file', f)
    continue

In [ ]:
#@title Save the processed signatures to file
# Writing dataset to pickle file
TMIDI.Tegridy_Pickle_File_Writer(CLASS_DATA, '/content/ClassySigsDataset.pickle')

In [ ]:
#@title Load your MIDI file to classify
full_path_to_MIDI_file = "/content/Input/clean_midi/10,000 Maniacs/A Campfire Song.mid" #@param {type:"string"}

MIDI_File = full_path_to_MIDI_file
SIG_S, SIG_F, ALL_F, PRL = ClassyMIDI.get_signatures(MIDI_File)

In [ ]:
#@title Classify your MIDI
print('Matching MIDI', MIDI_File)
print('MIDI Simple Signature', SIG_S)
print('MIDI Full Signature', SIG_F)

RATINGS_LIST = []

for i in range(1, len(CLASS_DATA)):
  RATINGS_LIST.append(fuzz.ratio(SIG_S, CLASS_DATA[i][3]))

MAX_idx = max(RATINGS_LIST)  
MATCH_idx = RATINGS_LIST.index(MAX_idx)

print('Best simple match rating is', MAX_idx)
print('Best simple match is', CLASS_DATA[MATCH_idx])

RATINGS_LIST = []

for i in range(1, len(CLASS_DATA)):
  RATINGS_LIST.append(fuzz.ratio(SIG_F, CLASS_DATA[i][4]))

MAX_idx = max(RATINGS_LIST)  
MATCH_idx = RATINGS_LIST.index(MAX_idx)
print('Best precise rating is', MAX_idx)
print('Best precise match is', CLASS_DATA[MATCH_idx])